# Results for TSMCN-8-L-229 with SNR=inf. Noise added test data is analyzed after experimental classification

RSAT=0.01 and 90 spectra per class

In [1]:
%matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.ticker import MaxNLocator
import numpy as np
from numpy import asarray
import pandas as pd
import math
import seaborn as sns  #heat map
import glob # batch processing of images


import matplotlib.font_manager as fm
import random
import sys
import os

from sklearn.datasets import make_regression
import tensorflow as tf
from sklearn.metrics import confusion_matrix    #confusion matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Collect all the font names available to matplotlib
font_names = [f.name for f in fm.fontManager.ttflist]
# print(font_names)

from scipy import signal
from scipy import interpolate

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve 
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score


from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RBF

#Sklearn model saving and loading
from joblib import dump, load

if '../../' not in sys.path:
    sys.path.append('../../')

from aimos.spectral_datasets.THz_datasets import THz_data

from aimos.misc.utils import simple_plotter


#Set random seed
os.environ['PYTHONHASHSEED'] = str(42)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
tf.random.set_global_generator(42)
tf.random.set_seed(42)  
# tf.random.get_global_generator().reset_from_seed(42)
np.random.seed(42)
random.seed(42)

from oneida import THz_mixture_data
from oneida_utils import concentrations_to_one_hot_encode, create_mixture_names
from oneida_utils import simple_spectrum_fig, simple_plot_raw_scores, plot_spectrum_with_scores, multiclass_roc_auc_score, multiclass_sensitivity_specificity_score, multiclass_sensitivity_threshold_score
from oneida_scoring_tools import calc_AMCAS, is_cui_present, is_cui_present_in_mult
from aimos.misc.utils import classifier_internals
from aimos.misc.utils import clf_post_processor
from oneida_utils import mixture_names_to_one_hot_encode
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from oneida_grad_cam import grad_cam

from stats import stats
stats(n_compounds=8)

2022-11-11 00:28:39.766171: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-11 00:28:39.877550: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-11 00:28:39.881801: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2022-11-11 00:28:39.881814: I tensorflow/stream_execu

Total 1-C combinations: 8
Total 2-C combinations: 28
Total 3-C combinations: 56
Total 4-C combinations: 70
Total 5-C combinations: 56
Total 6-C combinations: 28
Total 7-C combinations: 8
Total 8-C combinations: 1
Total combinations: 255


# Retrieve training(D)+ validation(V) mixtures and its variables

In [2]:
# initialize
TAAT = 0.001 
ASAT=0.01
RSAT=0.01

m = THz_mixture_data(resolution=0.016, pressure='1 Torr', verbosity=False)
m.initiate_THz_mixture_data(TAAT = TAAT, 
                               ASAT=ASAT, 
                               RSAT=RSAT)

reduced_labels = m.labels
reduced_labels.remove('')
reduced_labels.remove(' ')
reduced_labels.remove('Diluent')
print('reduced_labels', reduced_labels)


# data_filename = "datasets/TSMCN-5-L-229_DV_04-09-2022_time_22-26-37.pkl"
data_filename = "datasets/TSMCN-8-L-229_DV__TAAT_0.001_ASAT_0.01_RSAT_0.01_20-10-2022_time_23-16-29_class_cnt_90.pkl"
DV = pd.read_pickle(data_filename)
y = DV['y'].to_numpy()
mixture_names = DV['mixture_names'].to_numpy()
y_concentrations = DV[['y_c0', 'y_c1', 'y_c2','y_c3', 'y_c4', 'y_c5', 'y_c6', 'y_c7']].to_numpy()
X = DV.drop(['y','mixture_names', 'y_c0', 'y_c1', 'y_c2','y_c3', 'y_c4', 'y_c5', 'y_c6', 'y_c7'],axis=1).to_numpy()

final_neuron_number = np.unique(y, axis=0).shape[0]
print('Number of neurons in the final layer :', final_neuron_number)

print('labels from class:', m.labels)

Components :  [[0 1 2 3 4 5 6 7 8]]
Components shape :  (1, 9)
TAAT =  0.001
ASAT =  0.01
RSAT =  0.01
reduced_labels ['$C_2H_5OH$', '$CH_3CHO$', '$CH_3Cl$', '$CH_3CN$', '$CH_3OH$', '$H_2CO$', '$HCOOH$', '$HNO_3$']
Number of neurons in the final layer : 255
labels from class: ['$C_2H_5OH$', '$CH_3CHO$', '$CH_3Cl$', '$CH_3CN$', '$CH_3OH$', '$H_2CO$', '$HCOOH$', '$HNO_3$']


# preview one test mixture spectra using simple plotter

In [3]:
# idx = 239
# simple_plotter(m.frequencies,X[idx],linewidth=0.5,color='black',label=mixture_names[idx], 
#                    majorsize=6,minorsize=2,width=1, labelsize=8,legendsize=3, legendloc=2,  
#                    labelpad=4,fontsize='medium',fontweight='bold',
#                   xmajormplloc=0.5,xminormplloc=0.2, tickdirection='out')

# print(y_concentrations[idx])
# print(reduced_labels)

In [4]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(mixture_names)

mixture_types=le.classes_
# print(mixture_types)

In [5]:
#split intro train and validation set

#seeds used 123,237, 786
from sklearn.model_selection import train_test_split

global_indices=range(0, X.shape[0])
print(global_indices)

# (np.expand_dims(X,-1)
TRAIN_SIZE=0.60
VAL_SIZE=1-TRAIN_SIZE

x_train, x_val, y_train, y_val, train_indices, val_indices = train_test_split(np.expand_dims(X, axis=-1), y, global_indices, train_size=TRAIN_SIZE,
                                                   test_size=VAL_SIZE,
                                                   random_state=786,
                                                    stratify=y

                                                   )

print('X_train shape:', x_train.shape)
print('y_ohe_train shape:', y_train.shape)

print('X_val shape:', x_val.shape)
print('y_ohe_val shape:', y_val.shape)


print("All:", np.bincount(y) / float(len(y))*100  )
print("Training:", np.bincount(y_train) / float(len(y_train))*100  )
print("Validation:", np.bincount(y_val) / float(len(y_val))*100  )


range(0, 22950)
X_train shape: (13770, 229, 1)
y_ohe_train shape: (13770,)
X_val shape: (9180, 229, 1)
y_ohe_val shape: (9180,)
All: [0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215

In [6]:
from oneida_model import get_callbacks, get_optimizer, compile_and_fit, TSMCN_12_L_229

In [7]:
# model = TSMCN_12_L_229()

In [8]:
# tf.keras.utils.plot_model(model, to_file="RESULTS/TSMCN_8_L_229.png", show_shapes=True, rankdir='TB', dpi=150)

In [9]:
# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath='checkpoints/',
#     save_weights_only=True,
#     monitor='val_accuracy',
#     mode='max',
#     save_best_only=True)

model_name = data_filename.split('.pkl')[0].split('/')[1]
print(model_name)

TSMCN-8-L-229_DV__TAAT_0.001_ASAT_0.01_RSAT_0.01_20-10-2022_time_23-16-29_class_cnt_90


In [10]:
# # run on CPU for reproducibility, best epoch is 4.
# # with tf.device('/CPU:0'):
# #      stop_early = tf.keras.callbacks.EarlyStopping(monitor='SparseCatCrossentropy', patience=5)
# #     history = model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test), callbacks=[stop_early])

# # LambdaCallback(on_epoch_end=lambda batch, logs: print(model.layers[0].get_weights()))
# with tf.device('/CPU:0'):
#     stop_early = tf.keras.callbacks.EarlyStopping(monitor='SparseCatCrossentropy', patience=5)
#     history = model.fit(x_train, y_train, epochs=30, validation_data=(x_val, y_val), 
#                         callbacks=[model_checkpoint_callback, stop_early]
# )



In [11]:
# #Sklearn model saving and loading
# from joblib import dump, load
# import datetime
# from datetime import date, datetime
# now = datetime.now()
# dt_string = now.strftime("%d-%m-%Y_time_%H-%M-%S")
# model.save('checkpoints/' + model_name + '_' + dt_string + '.hdf5')

In [12]:
model = tf.keras.models.load_model('model/TSMCN-8-L-229_DV__TAAT_0.001_ASAT_0.01_RSAT_0.01_20-10-2022_time_23-16-29_class_cnt_90_20-10-2022_time_23-30-09.hdf5', compile=False)


2022-11-11 00:29:05.178687: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2022-11-11 00:29:05.178861: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2022-11-11 00:29:05.178992: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2022-11-11 00:29:05.182448: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuso

In [13]:
# model.save('model/' + model_name + '_' + dt_string + '.hdf5')
# history = np.load('model/TSMCN-8-L-229_DV__TAAT_0.001_ASAT_0.01_RSAT_0.05_21-09-2022_time_18-53-02_13-10-2022_time_23-21-11history.npy',)
# np.load('model/' + model_name + '_' + dt_string + 'epoch' + '.npy',history.epoch)
history=np.load('model/TSMCN-8-L-229_DV__TAAT_0.001_ASAT_0.01_RSAT_0.01_20-10-2022_time_23-16-29_class_cnt_90_20-10-2022_time_23-30-09history.npy',allow_pickle='TRUE').item()
epochs=np.load('model/TSMCN-8-L-229_DV__TAAT_0.001_ASAT_0.01_RSAT_0.01_20-10-2022_time_23-16-29_class_cnt_90_20-10-2022_time_23-30-09epoch.npy',allow_pickle='TRUE')



In [14]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(x_val)

287/287 [==============================] - 1s 3ms/step


In [15]:
y_ohe = concentrations_to_one_hot_encode(y_concentrations).astype('int64')
y_train_ohe = y_ohe[train_indices]
y_val_ohe = y_ohe[val_indices]
y_train_ohe_tensor = tf.convert_to_tensor(y_train_ohe, np.int64)
y_val_ohe_tensor = tf.convert_to_tensor(y_val_ohe, np.int64)

In [16]:
# Analyze weights

model.get_weights()[9].shape

(255,)

In [17]:
# DV_noisy['y_c8']=DV['y_c8']
# from scipy import fftpack
# def fft_filter(frequencies, absorbances, factor=100):
#     sig=absorbances
#     time_vec=frequencies
#     period = 1.
#     # The FFT of the signal
#     sig_fft = fftpack.fft(sig)

#     # And the power (sig_fft is of complex dtype)
#     power = np.abs(sig_fft)**2

#     # The corresponding frequencies
#     sample_freq = fftpack.fftfreq(sig.size, d=1.0)



#     # Find the peak frequency: we can focus on only the positive frequencies
#     pos_mask = np.where(sample_freq > 0)
#     freqs = sample_freq[pos_mask]
#     peak_freq = freqs[power[pos_mask].argmax()]

#     # Check that it does indeed correspond to the frequency that we generate
#     # the signal with
#     np.allclose(peak_freq, 1./period)



#     # scipy.signal.find_peaks_cwt can also be used for more advanced
#     # peak detection

#     print('peak_freq', peak_freq)
#     sel_peak_freq = factor*peak_freq
#     high_freq_fft = sig_fft.copy()
#     high_freq_fft[np.abs(sample_freq) > sel_peak_freq] = 0
#     filtered_sig = fftpack.ifft(high_freq_fft)



## comparing the 4-component mixture with simulation

In [18]:
# target_index = 344665
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     shortDF=DV[DV['y']==19]
# #     display(shortDF['y_c4'])
#     display(shortDF)

In [19]:
# print(reduced_labels)
# special_df = DV[DV.index==344665]
# display(special_df)

# x_special = special_df.drop(['y','mixture_names', 'y_c0', 'y_c1', 'y_c2','y_c3', 'y_c4', 'y_c5', 'y_c6', 'y_c7'],axis=1).to_numpy()


In [20]:
# DV_noisy['y_c8']=DV['y_c8']
# exp_path = '../../data/Mixture_exp_data/'
# exp_filepath = '4 Comp-mix_ 67% CH3OH + 30% CH3CHO + 2% CH3Cl + 1% CH3CN/0.67 CH3OH + 0.3 CH3CHO + 0.02 CH3Cl + 0.01 CH3CN - v2.xlsx'



# all_unique_labels= mixture_types

# df_exp1 = pd.read_excel(exp_path + exp_filepath)
# abs_exp1 = df_exp1[df_exp1.columns[1]].to_numpy()

# indices = abs_exp1 >0
# pos_abs=abs_exp1*indices


In [21]:

# filtered_abs_resampled = signal.resample(pos_abs, len(m.frequencies))



# indices = filtered_abs_resampled >0
# filtered_abs_resampled=filtered_abs_resampled*indices
# expanded_abs = np.expand_dims(np.expand_dims(filtered_abs_resampled, axis=-1), axis=0)
# pred_exp_label = np.argmax(model.predict(expanded_abs), axis=-1)[0]


# print('predicted index ', pred_exp_label)
# print('predicted label', mixture_types[pred_exp_label])

# plt.figure(figsize=(16,9),dpi=200)
# fig,axs = plt.subplots(2,1, figsize=(16,9),dpi=200)
# # plt.plot(pos_abs)
# axs[0].plot(filtered_abs_resampled, label='resampled postive exp spectra\n0.67 CH3OH + 0.3 CH3CHO + 0.02 CH3Cl + 0.01 CH3CN\n model predicts CH3CN + CH3OH')
# axs[0].plot(np.squeeze(x_special), label='simulation\n 0.58 CH3OH + 0.29 CH3CHO + 0.05 CH3Cl + 0.015 CH3CN')
# axs[0].set_ylim(0,0.4)
# axs[0].legend()

# axs[1].plot(abs_exp1, color='black', label='original exp spectra\n0.67 CH3OH + 0.3 CH3CHO + 0.02 CH3Cl + 0.01 CH3CN\n model predicts CH3CN + CH3OH')
# axs[1].legend()


# # fig.savefig(r'RESULTS/filtered_spectra/' + '0.67 CH3OH + 0.3 CH3CHO + 0.02 CH3Cl + 0.01 CH3CN - v2' + '_comparison.png', bbox_inches='tight')

## filtering and classifying

In [22]:
import scipy
from scipy.signal import savgol_filter
from oneida_filters import FFT_filter

In [23]:
# DV_noisy['y_c8']=DV['y_c8']


In [24]:
import dataframe_image as dfi
# df = pd.DataFrame(np.random.randn(6, 6), columns=list('ABCDEF'))
# df_styled = df.style.background_gradient() #adding a gradient based on values in cell
# dfi.export(df_styled,"mytable.png")

In [26]:

# exps = ['2 Comp-mix_ 30 % CH3Cl - 70% CH3CN/Mix 50% Dilute CM-ACN.xlsx',
# '2 Comp-mix_ 30 % CH3Cl - 70% CH3CN/Pure Mix CM-ACN.xlsx',
# '2 Comp-mix_ 30 % CH3Cl - 70% CH3CN/Mix 90% Dilute CM-ACN.xlsx',
# '3 Comp-mix_ 90+% CH3OH + 5-% CH3CN + 5-% CH3CL/0.9 CH3OH + 0.05 CH3CN + 0.05 CH3Cl - 1.xlsx',
# '3 Comp-mix_ 90+% CH3OH + 5-% CH3CN + 5-% CH3CL/0.9 CH3OH + 0.05 CH3CN + 0.05 CH3Cl - 2.xlsx',
# '4 Comp-mix_ 67% CH3OH + 30% CH3CHO + 2% CH3Cl + 1% CH3CN/0.67 CH3OH + 0.3 CH3CHO + 0.02 CH3Cl + 0.01 CH3CN - v2.xlsx',
# '4 Comp-mix_ 67% CH3OH + 30% CH3CHO + 2% CH3Cl + 1% CH3CN/90% Dilute in N2 - 0.67 CH3OH + 0.3 CH3CHO + 0.02 CH3Cl + 0.01 CH3CN'       ]

# true_label=[81,81,81,82,82,19,19]

exps = ['2 Comp-mix_ 30 % CH3Cl - 70% CH3CN/Mix 50% Dilute CM-ACN.xlsx',
'2 Comp-mix_ 30 % CH3Cl - 70% CH3CN/Pure Mix CM-ACN.xlsx',
'2 Comp-mix_ 30 % CH3Cl - 70% CH3CN/Mix 90% Dilute CM-ACN.xlsx',
'3 Comp-mix_ 90+% CH3OH + 5-% CH3CN + 5-% CH3CL/0.9 CH3OH + 0.05 CH3CN + 0.05 CH3Cl - 1.xlsx',
'3 Comp-mix_ 90+% CH3OH + 5-% CH3CN + 5-% CH3CL/0.9 CH3OH + 0.05 CH3CN + 0.05 CH3Cl - 2.xlsx',
'4 Comp-mix_ 67% CH3OH + 30% CH3CHO + 2% CH3Cl + 1% CH3CN/0.67 CH3OH + 0.3 CH3CHO + 0.02 CH3Cl + 0.01 CH3CN - v2.xlsx',
]

true_label=[81,81,81,82,82,19]

exp_path = '../../data/Mixture_exp_data/'
exp_filepath = '4 Comp-mix_ 67% CH3OH + 30% CH3CHO + 2% CH3Cl + 1% CH3CN/0.67 CH3OH + 0.3 CH3CHO + 0.02 CH3Cl + 0.01 CH3CN - v2.xlsx'


def classify_exp(exp_path,exp_filepath,mixture_types,true_label, excel=True):
    all_unique_labels= mixture_types
    if excel:
        df_exp1 = pd.read_excel(exp_path + exp_filepath)
    else:
        df_exp1 = pd.read_csv(exp_path + exp_filepath)


    freq_exp1 = df_exp1[df_exp1.columns[0]].to_numpy()
    abs_exp1 = df_exp1[df_exp1.columns[1]].to_numpy()
    
#     fft_filter(freq_exp1, abs_exp1, factor=75)

    dfy_resampled= signal.resample(abs_exp1, len(m.frequencies))
    dfx_resampled= signal.resample(freq_exp1, len(m.frequencies))
    expanded_abs = np.expand_dims(np.expand_dims(dfy_resampled, axis=-1), axis=0)
    pred_exp_label = np.argmax(model.predict(expanded_abs), axis=-1)[0]
    
    score_predictions = probability_model.predict(expanded_abs)
    scoreDf = pd.DataFrame()
    scoreDf['species'] = mixture_types
#     scoreDf['Label'] = scoreDf.index
    scoreDf['Species present'] = mixture_types
    scoreDf.drop('species',axis=1, inplace=True)
#     import pdb; pdb.set_trace()
    scoreDf['Score'] = score_predictions[0]
#     scoreDf.to_csv(r'RESULTS/exp_preds/' + 'exp' + str(idx) + '.csv')
    
    sorted_scoresDf = scoreDf.sort_values(by='Score', ascending=False).head(n=5)
    
#     print(score_predictions)
    print('Experiment name: ',exp_filepath.split('/')[0])
    print('File name: ',exp_filepath.split('/')[1])
    print('predicted index ', pred_exp_label)
    print('predicted label', mixture_types[pred_exp_label])
#     display(sorted_scoresDf)
    print('-----------------------------------------------------')
    

    return pred_exp_label, sorted_scoresDf
idx = 0 
for experiment in exps:
    pred_exp_label, sorted_scoresDf = classify_exp(exp_path,experiment,mixture_types, true_label[idx],excel=True)
    
#     sorted_scoresDf.style.set_properties(**{'background-color': 'white',
#                            'color': 'blue'})
    
    display(sorted_scoresDf.style.hide_index())
#     df_styled = sorted_scoresDf.style.background_gradient(cmap='Blues',low=0, high=0)
#     dfi.export(df_styled,"result_df_" + str(idx) + ".png")
    dfi.export(sorted_scoresDf.style.hide_index(), r"RESULTS/results_figures/result_df_" + str(idx) + ".png", table_conversion='matplotlib',dpi=150)
    idx+=1

1/1 [==============================] - 0s 28ms/step
Experiment name:  2 Comp-mix_ 30 % CH3Cl - 70% CH3CN
File name:  Mix 50% Dilute CM-ACN.xlsx
predicted index  81
predicted label $CH_3Cl$+$CH_3CN$
-----------------------------------------------------


/tmp/ipykernel_58735/2175816645.py:72: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  display(sorted_scoresDf.style.hide_index())


Species present,Score
$CH_3Cl$+$CH_3CN$,0.999458
$C_2H_5OH$+$CH_3Cl$+$CH_3CN$,0.000540
$CH_3Cl$+$CH_3CN$+$CH_3OH$,0.000002
$C_2H_5OH$+$CH_3Cl$+$CH_3CN$+$HCOOH$,0.000000
$CH_3Cl$+$CH_3CN$+$HCOOH$,0.000000


/tmp/ipykernel_58735/2175816645.py:75: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  dfi.export(sorted_scoresDf.style.hide_index(), r"RESULTS/results_figures/result_df_" + str(idx) + ".png", table_conversion='matplotlib',dpi=150)


1/1 [==============================] - 0s 24ms/step
Experiment name:  2 Comp-mix_ 30 % CH3Cl - 70% CH3CN
File name:  Pure Mix CM-ACN.xlsx
predicted index  81
predicted label $CH_3Cl$+$CH_3CN$
-----------------------------------------------------


...

/tmp/ipykernel_58735/2175816645.py:75: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  dfi.export(sorted_scoresDf.style.hide_index(), r"RESULTS/results_figures/result_df_" + str(idx) + ".png", table_conversion='matplotlib',dpi=150)


1/1 [==============================] - 0s 32ms/step
Experiment name:  4 Comp-mix_ 67% CH3OH + 30% CH3CHO + 2% CH3Cl + 1% CH3CN
File name:  0.67 CH3OH + 0.3 CH3CHO + 0.02 CH3Cl + 0.01 CH3CN - v2.xlsx
predicted index  82
predicted label $CH_3Cl$+$CH_3CN$+$CH_3OH$
-----------------------------------------------------


/tmp/ipykernel_58735/2175816645.py:72: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  display(sorted_scoresDf.style.hide_index())


Species present,Score
$CH_3Cl$+$CH_3CN$+$CH_3OH$,0.985995
$CH_3Cl$+$CH_3CN$+$CH_3OH$+$HNO_3$,0.012360
$CH_3CN$+$CH_3OH$,0.001638
$CH_3Cl$+$CH_3CN$+$CH_3OH$+$HCOOH$,0.000007
$CH_3CN$+$CH_3OH$+$HNO_3$,0.000000


/tmp/ipykernel_58735/2175816645.py:75: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  dfi.export(sorted_scoresDf.style.hide_index(), r"RESULTS/results_figures/result_df_" + str(idx) + ".png", table_conversion='matplotlib',dpi=150)


# classify some pure spectra

In [27]:


exps = [
       '1 comp/Methanol 1 Torr WDL 8.csv',
       '1 comp/Formic Acid 1 Torr WDL 5 Notch Shift.csv',
       '1 comp/Acetaldehyde 1 Torr WDL 6 Notch.csv',
       
       
       '1 comp/1-Sweep CH3Cl 220-330-1 Torr.csv',
       
       '1 comp/1-Sweep 0p5 Torr-CH3CN.csv',]
true_label=[112,248,0,80,64]

exp_path = '../../data/Mixture_exp_data/'
exp_filepath = '4 Comp-mix_ 67% CH3OH + 30% CH3CHO + 2% CH3Cl + 1% CH3CN/0.67 CH3OH + 0.3 CH3CHO + 0.02 CH3Cl + 0.01 CH3CN - v2.xlsx'


def classify_exp(exp_path,exp_filepath,mixture_types,true_label, excel=True):
    all_unique_labels= mixture_types
    if excel:
        df_exp1 = pd.read_excel(exp_path + exp_filepath)
    else:
        df_exp1 = pd.read_csv(exp_path + exp_filepath)


    freq_exp1 = df_exp1[df_exp1.columns[0]].to_numpy()
    abs_exp1 = df_exp1[df_exp1.columns[1]].to_numpy()
    
#     fft_filter(freq_exp1, abs_exp1, factor=75)

    dfy_resampled= signal.resample(abs_exp1, len(m.frequencies))
    dfx_resampled= signal.resample(freq_exp1, len(m.frequencies))
    expanded_abs = np.expand_dims(np.expand_dims(dfy_resampled, axis=-1), axis=0)
    pred_exp_label = np.argmax(model.predict(expanded_abs), axis=-1)[0]
    
    score_predictions = probability_model.predict(expanded_abs)
    scoreDf = pd.DataFrame()
    scoreDf['species'] = mixture_types
#     scoreDf['Label'] = scoreDf.index
    scoreDf['Species present'] = mixture_types
    scoreDf.drop('species',axis=1, inplace=True)
#     import pdb; pdb.set_trace()
    scoreDf['Score'] = score_predictions[0]
#     scoreDf.to_csv(r'RESULTS/exp_preds/' + 'exp' + str(idx) + '.csv')
    
    sorted_scoresDf = scoreDf.sort_values(by='Score', ascending=False).head(n=5)
    
#     print(score_predictions)
    print('Experiment name: ',exp_filepath.split('/')[0])
    print('File name: ',exp_filepath.split('/')[1])
    print('predicted index ', pred_exp_label)
    print('predicted label', mixture_types[pred_exp_label])
    display(sorted_scoresDf.style.hide_index())
    dfi.export(sorted_scoresDf.style.hide_index(), r"RESULTS/results_figures/result_pure_df_" + str(idx) + ".png", table_conversion='matplotlib',dpi=150)
    print('-----------------------------------------------------')
    

    return pred_exp_label
idx = 0 
for experiment in exps:
    classify_exp(exp_path,experiment,mixture_types, true_label[0],excel=False)
    
    idx+=1

1/1 [==============================] - 0s 34ms/step
Experiment name:  1 comp
File name:  Methanol 1 Torr WDL 8.csv
predicted index  112
predicted label $CH_3OH$


/tmp/ipykernel_58735/1398306016.py:51: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  display(sorted_scoresDf.style.hide_index())


Species present,Score
$CH_3OH$,0.999994
$CH_3CHO$+$CH_3CN$+$CH_3OH$,0.000006
$CH_3CN$+$CH_3OH$,0.000000
$C_2H_5OH$+$CH_3CN$+$CH_3OH$,0.000000
$CH_3Cl$+$CH_3OH$,0.000000


-----------------------------------------------------


/tmp/ipykernel_58735/1398306016.py:52: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  dfi.export(sorted_scoresDf.style.hide_index(), r"RESULTS/results_figures/result_pure_df_" + str(idx) + ".png", table_conversion='matplotlib',dpi=150)


...

1/1 [==============================] - 0s 25ms/step
Experiment name:  1 comp
File name:  1-Sweep 0p5 Torr-CH3CN.csv
predicted index  64
predicted label $CH_3CN$


/tmp/ipykernel_58735/1398306016.py:51: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  display(sorted_scoresDf.style.hide_index())


Species present,Score
$CH_3CN$,0.998151
$CH_3CN$+$CH_3OH$,0.001848
$CH_3Cl$+$CH_3CN$,0.000000
$CH_3Cl$+$CH_3CN$+$CH_3OH$,0.000000
$C_2H_5OH$+$CH_3CN$,0.000000


-----------------------------------------------------


/tmp/ipykernel_58735/1398306016.py:52: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  dfi.export(sorted_scoresDf.style.hide_index(), r"RESULTS/results_figures/result_pure_df_" + str(idx) + ".png", table_conversion='matplotlib',dpi=150)


In [39]:
fig = plot_spectrum_with_scores(np.squeeze(x_val), y_val, predictions, m.frequencies,mixture_types, 19, dpi = 300)
